# From list-of-ORs to rough risk prediction

This notebook explores how to convert allelic odds ratios, reported by many publications, to genotypic relative risk, with the goal of combining relative risks for an estimate of relative risk for combined genotypes.

This is specifically inspired by data from the paper [Schumacher et al. 2018](https://www.nature.com/articles/s41588-018-0142-8), but is likely to be applicable to using data from similar research reports.

## 0. Big assumption: Multiplicative model

Schumacher et al. 2018 uses a multiplicative model to combine odds ratios for multiple alleles different locations. This is also called "log additive", as an operation that sums the log of numbers is equivalent to one that multiplies them. (See the appendix section "Proportion of familial risk explained.")

This means, for a position with potential alleles A and B, the relative risks associated with A and B (`rel_A` and `rel_B`) can be multiplied together to calculate combined risk:

* relative risk for genotype AA: `rel_AA = rel_A * rel_A`
* relative risk for genotype AB = `rel_AB = rel_A * rel_B`
* relative risk for genotype BB = `rel_BB = rel_B * rel_B`

In addition, this can be done across positions, e.g.

* relative risk for "AA at position 1, AA at position 2" = `rel_1_AA * rel_2_AA`

**The multiplicative model is a major assumption.** The reality is that biology is entangled: for example, the product of one gene might be used by another. This model is an approximation that assumes the effects of different gene variants are independent of each other – but it's likely that this isn't always true.

## 1. Starting from the end: relative risks

Let's start with a couple hypothetical variables:

* `risk_D` = overall frequency of "disease d"
* `freq_A` = frequency of "allele A"
* `rel_A` = relative risk of "disease d" associated with "allele A" (relative to average risk, **risk_D**)

I'm going to play with these – you can change them and run code again to see how it affects other things.

In [1]:
risk_D = 0.1  # between 0 and 1
freq_A = 0.3  # between 0 and 1
rel_A = 1.2   # larger than 0 (1 = equal to average risk, 2 = 2x average risk, etc.)

### 1.1 Relative risks for allele A and B

Although the **odds ratio** is typically easier to derive from observational data, it's not a very intuitive metric for describing risk.

**Relative risk** has a more clear definition – how the risk of a condition within context A compares to context B. For example, for two potential alleles a position, "A" and "B"...

* `risk_A` = risk of disease for "allele A"
* `risk_B` = risk of disease for "allele B"
* `risk_D` = risk of disease for the general population

Then `rel_A = risk_A / risk_G` is a relative risk – the risk given condition A, relative to average risk. And `rel_B = risk_B / risk_G` is a relative risk for B.

Because all alleles for this position are either "A" or "B", we expect these to balance. A frequency-weighted sum of `rel_A` and `rel_B` should equal 1.0 – that is, equal to the average risk for the general population.

`freq_A * rel_A + freq_B * rel_B = 1.0`

That means, given `freq_A` and `rel_A`, we can determine `freq_B` and `rel_B`:

In [2]:
freq_B = 1 - freq_A                  # allele frequency for B
rel_B = (1 - (freq_A * rel_A)) / freq_B    # relative risk of D for B

import json
print(json.dumps({
    'freq_A': freq_A,
    'rel_A': rel_A,
    'freq_B': freq_B,
    'rel_B': rel_B},
    sort_keys=True, indent=2))

{
  "freq_A": 0.3,
  "freq_B": 0.7,
  "rel_A": 1.2,
  "rel_B": 0.9142857142857144
}


## 1.2 Relative risks for genotypes

Actually, people each have two alleles!

Let's check our work. Using allele frequencies, we can calculate the frequency of each genotype. And using the multiplicative model, we can calculate the relative risk associated with each genotype.

We expect the weighted sum to be 1. Is it?

In [3]:
# Hardy–Weinberg!
freq_AA = freq_A * freq_A
freq_AB = 2 * freq_A * freq_B
freq_BB = freq_B * freq_B

rel_AA = rel_A * rel_A
rel_AB = rel_A * rel_B
rel_BB = rel_B * rel_B

print(json.dumps({
    'freq_AA': freq_AA,
    'freq_AB': freq_AB,
    'freq_BB': freq_BB,
    'rel_AA': rel_AA,
    'rel_AB': rel_AB,
    'rel_BB': rel_BB,
    }, sort_keys=True, indent=2))

weighted_sum = freq_A**2 * rel_A**2 + 2 * freq_A * freq_B * rel_A * rel_B + freq_B**2 * rel_B**2

print('\n' + 'weighted sum: {}'.format(weighted_sum))

{
  "freq_AA": 0.09,
  "freq_AB": 0.42,
  "freq_BB": 0.48999999999999994,
  "rel_AA": 1.44,
  "rel_AB": 1.0971428571428572,
  "rel_BB": 0.8359183673469389
}

weighted sum: 1.0


# 2. Calculating an odds ratio

For the following four statistics:

* `A_wD` = number of times allele A is found with disease
* `A_woD` = number of times allele A is found without disease
* `B_wD` = number of times allele B is found with disease
* `B_woD` = number of times allele B is found without disease

Then the odds ratio of A vs. B for disease D is:

`OR = (A_wD / A_woD) / (B_wD / B_woD)`

Publications often report odds ratios because the metric can be determined from observations of diseased and disease-free populations – and doesn't require determining the overall frequency of disease (`risk_D`) in the population. (That number can be tricky to determine!)

But if we _did_ know `risk_D` – and the allele frequencies and relative risks associated with each allele – we could predict what odds ratios we would expect to see.

In [4]:
def calc_OR(D, fA, rA):

    # Calculate B allele values.
    fB = 1 - fA
    rB = (1 - (fA * rA)) / fB

    # Calculate for each genotype.
    AA_wD = (fA**2) * (D * rA**2)         # % of population that is AA and has D
    AA_woD = (fA**2) * (1 - D * rA**2)    # % of population that is AA and doesn't have D
    AB_wD = (2 * fA * fB) * (D * rA * rB)
    AB_woD = (2 * fA * fB) * (1 - D * rA * rB)
    BB_wD = (fB**2) * (D * rB**2)
    BB_woD = (fB**2) * (1 - D * rB**2)
    
    # Use genotype numbers to calculate for each allele.
    A_wD = 2 * AA_wD + AB_wD     # fraction of A alleles in someone with D. Count 2x alleles in homozygous folks!
    A_woD = 2 * AA_woD + AB_woD
    B_wD = 2 * BB_wD + AB_wD
    B_woD = 2 * BB_woD + AB_woD

    # Raise an error if unnatural numbers exist: the combined input can't exist.
    for var in [AA_wD, AA_woD, AB_wD, AB_woD, BB_wD, BB_woD]:
        if var <= 0.0 or var >= 1.0:
            raise ValueError
    for var in [A_wD, A_woD, B_wD, B_woD]:
        if var <= 0.0 or var >= 2.0:
            raise ValueError
    
    OR = (A_wD / A_woD) / (B_wD / B_woD)    
    return(OR)

calc_OR(risk_D, freq_A, rel_A)

1.3551136363636362

# 3. Calculating "relative risks" for a given "odds ratio"

As demonstrated above, given the following:

1. an estimate of disease frequency (`risk_D`)
2. frequency of an allele (`freq_A`)
3. an odds ratio associated with it (`rel_A`)

It's possible to calculate the odds ratio associated with A.

But we _have_ the odds ratio already. Let's presume that is what we have:

1. an odds ratio for A (`OR`)
2. an estimate of disease frequency (`risk_D`)
3. frequency of an allele (`freq_A`)

…and what we _want_ is `rel_A`.

If we were great at rearranging variables, we could figure out the equation to solve this exactly. I'm not. In the interest of solving problems, I'll make a hack...

## 3.1 Build a look-up dict

I'll use the equation I already have, run it for all potential inputs, and build a look-up dict, `rel_A_calc`.

`rel_A_calc[risk_D][freq_A][OR] = rel_A`

In [5]:
rel_A_calc = dict()
or_diff = dict()

# Range from 0.01 to 0.99 allele frequency
for i in range(1,100):

    # Range from 1.001 to 1.999 relative risk
    for j in range(1,2000):

        # Range from 0.01 to 0.99 disease risk
        for k in range(1,100):
            freq_A = i * 1.0 / 100
            rel_A = 1 + j * 1.0 / 1000
            risk_D = k * 1.0 / 100

            # Sometimes the combination is impossible.
            try: 
                oddsrat = calc_OR(risk_D, freq_A, rel_A)
            except ValueError:
                continue

            # Store OR to the closest one rounded by hundreth
            or_rounded = round(oddsrat, 2)
            if or_rounded == 1.00:
                continue
            or_difference = abs(or_rounded - oddsrat)

            if risk_D not in rel_A_calc:
                rel_A_calc[risk_D] = dict()
                or_diff[risk_D] = dict()
            if freq_A not in rel_A_calc[risk_D]:
                rel_A_calc[risk_D][freq_A] = dict()
                or_diff[risk_D][freq_A] = dict()

            if (or_rounded not in rel_A_calc[risk_D][freq_A] or
                    or_difference < or_diff[risk_D][freq_A][or_rounded]):
                rel_A_calc[risk_D][freq_A][or_rounded] = rel_A
                or_diff[risk_D][freq_A][or_rounded] = or_difference

## 3.2 Save to file

Let's store this in a file. (This file is a bit large. Maybe someone would like to optimize this.)

In [6]:
import gzip
with gzip.open('relrisk_dict.json.gz', 'wt') as f:
    json.dump(rel_A_calc, f, sort_keys=True)

## 3.3 Example lookup

Here's an example of looking up a "relative risk" estimate for allele A, given (1) estimate of disease prevalence `risk_D`, (2) allele frequency for A `freq_A`, and (3) odds ratio for allele A `or_a`.

In [7]:
risk_D = 0.3
freq_A = 0.25
or_a = 1.1

freq_B = 1 - freq_A

rel_A = rel_A_calc[risk_D][freq_A][or_a]
rel_B = (1 - (freq_A * rel_A)) / freq_B

print(json.dumps({'rel_A': rel_A, 'rel_B': rel_B}, sort_keys=True, indent=2))

{
  "rel_A": 1.05,
  "rel_B": 0.9833333333333334
}


# 4 Combining genotypes and locations

Combining relative risks to get a total risk estimate is fairly simple with the multiplicative model, and follows these two rules:

1. For each genotype, relative risk of each allele (as determined above) are multiplied together
2. Across locations, relative risk of the genotype at each location are multiplied together

When this procedure is done to estimate relative risks in a random set of people, the risk estimates should average out to 1. If not, we should wonder if something went wrong!